# This notebook explores the SKA1 SDP Design Equations

## Source: PDR05 v1.85

In [ ]:
""" These first few lines below import the IPython definitions and methods that we will use, including the ability to
display them nicely"""
from sympy import init_session
init_session(
    use_latex=True,
    use_unicode=True,
    pretty_print=True,
    quiet=False,
)
from parameter_definitions import ParameterContainer
# Currently this file below (design_equations.py) only contains variable definitions that are used for display purposes.
# All computations are performed in the notebook itself for now. This was the same way we initially did the approached 
# the imaging equations. When they become complex it is usually necessary and desirable to move the code from the notebook
# to the dedicated Python code files.
from design_equations import DesignEquations as de 
o = ParameterContainer()
de.define_symbolic_variables(o)  # o is updated in-place

from api_ipython import SkaIPythonAPI as iapi
from parameter_definitions import *
from parameter_definitions import Constants as c

verbose=False

# Switch for the faceting optimization
scale_predict_by_facet=False

### 1.	The peak FLOP capability of the units taken together has to exceed the total FLOPS required for spectral line and continuum processing combined 

In [ ]:
o.Ncu * o.RcuFLOP > o.RspecFLOP + o.RcontFLOP + o.RfastFLOP

In [ ]:
#Or numerically
result1_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.All, Bands.Low, bldta=True, on_the_fly=False
                                          , expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
result1_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid1, bldta=True, on_the_fly=False
                                          , expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
result1_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid2, bldta=True, on_the_fly=False
                                          , expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)

print '\nDesign Equation 1 evaluates to:\n'
print '%s > %.2f PetaFLOPS \tfor SKA1 LOW' % (str(o.Ncu * o.RcuFLOP), result1_low / c.peta)
print '%s > %.2f PetaFLOPS \tfor SKA1 MID (Band 1)' % (str(o.Ncu * o.RcuFLOP), result1_mid1 / c.peta)
print '%s > %.2f PetaFLOPS \tfor SKA1 MID (Band 2)' % (str(o.Ncu * o.RcuFLOP), result1_mid2 / c.peta)

### 2.	The total memory bandwidth required has to exceed the required memory bandwidth

In [ ]:
o.Ncu * o.RcuIo > o.Fci*(o.RspecFLOP + o.RcontFLOP + o.RfastFLOP)

In [ ]:
#Or numerically
print '\nDesign Equation 2 evaluates to:\n'
Fci = 2
result2_low = result1_low*Fci
result2_mid1 = result1_mid1*Fci
result2_mid2 = result1_mid2*Fci

print '%s > %.2f PetaBytes/s \tfor SKA1 LOW' % (str(o.Ncu * o.RcuIo), result2_low / c.peta)
print '%s > %.2f PetaBytes/s \tfor SKA1 MID (Band 1)' % (str(o.Ncu * o.RcuIo), result2_mid1 / c.peta)
print '%s > %.2f PetaBytes/s \tfor SKA1 MID (Band 2)' % (str(o.Ncu * o.RcuIo), result2_mid2 / c.peta)

### 3.	The visibility buffer of all compute units together must be sufficiently large to hold both the full resolution visibilities and frequency binned visibilities for continuum processing 

In [ ]:
o.Ncu * o.McuBuf > o.MspecBufVis + o.McontBufVis

In [ ]:
result3_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.ContAndSpectral, Bands.Low, bldta=True
                                           , on_the_fly=False, expression='Mbuf_vis', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
result3_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.ContAndSpectral, Bands.Mid1, bldta=True
                                           , on_the_fly=False, expression='Mbuf_vis', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
result3_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.ContAndSpectral, Bands.Mid2, bldta=True
                                           , on_the_fly=False, expression='Mbuf_vis', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)


#Or numerically
print '\nDesign Equation 3 evaluates to:\n'
Fci = 2
print '%s > %.2f PetaBytes \tfor SKA1 LOW' % (str(o.Ncu * o.McuBuf), result3_low / c.peta)
print '%s > %.2f PetaBytes \tfor SKA1 MID (Band 1)' % (str(o.Ncu * o.McuBuf), result3_mid1 / c.peta)
print '%s > %.2f PetaBytes \tfor SKA1 MID (Band 2)' % (str(o.Ncu * o.McuBuf), result3_mid2 / c.peta)

### 4.	The total bandwidth to the visibility buffer must exceed the greater of the bandwidth required for the continuum or spectral line case. If we assume that processing is limited by available FLOPs then the we can compute the rates as

In [ ]:
o.Ncu * o.RcuIo > Max(o.RspecIo*(o.RspecFLOP + o.RcontFLOP)/o.RspecFLOP, o.RcontIo*(o.RspecFLOP + o.RcontFLOP)/o.RcontFLOP)

In [ ]:
bldta = True
otf = False
Rflop_cont_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.Continuum, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rflop_spec_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.Spectral, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rio_cont_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.Continuum, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rio', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rio_spec_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.Spectral, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rio', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)

Rflop_cont_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Continuum, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rflop_spec_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Spectral, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rio_cont_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Continuum, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rio', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rio_spec_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Spectral, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rio', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)

Rflop_cont_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Continuum, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rflop_spec_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Spectral, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rio_cont_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Continuum, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rio', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rio_spec_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.Spectral, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rio', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)




result4_low = Max((Rio_cont_low/Rflop_cont_low)*(Rflop_cont_low + Rflop_spec_low), (Rio_spec_low/Rflop_spec_low)*(Rflop_cont_low + Rflop_spec_low))
result4_mid1 = Max((Rio_cont_mid1/Rflop_cont_mid1)*(Rflop_cont_mid1 + Rflop_spec_mid1), (Rio_spec_mid1/Rflop_spec_mid1)*(Rflop_cont_mid1 + Rflop_spec_mid1))

result4_mid2 = Max((Rio_cont_mid2/Rflop_cont_mid2)*(Rflop_cont_mid2 + Rflop_spec_mid2), (Rio_spec_mid2/Rflop_spec_mid2)*(Rflop_cont_mid2 + Rflop_spec_mid2))


#Or numerically
print '\nDesign Equation 4 evaluates to:\n'
print '%s > %.2f TB/s \tfor SKA1 LOW' % (str(o.Ncu * o.RcuIo), result4_low / c.tera)
print '%s > %.2f TB/s \tfor SKA1 MID (Band 1)' % (str(o.Ncu * o.RcuIo), result4_mid1 / c.tera)
print '%s > %.2f TB/s \tfor SKA1 MID (Band 2)' % (str(o.Ncu * o.RcuIo), result4_mid2 / c.tera)

### 5.	If frequency-polarisation-beam parallelism only is to be used with no faceting or other image plane or uv plane division then

In [ ]:
# Working memory per compute unit must be greater than the target grid memory
o.McuWork > o.MuvGrid

In [ ]:
bldta = True
otf = False
Npix_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.All, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Npix_linear', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Npix_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Npix_linear', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)

Npix_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Npix_linear', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)

result4_low = 16 * Npix_low**2 # Eq 9, in bytes
result4_mid1 = 16 * Npix_mid1**2 # Eq 9, in bytes
result4_mid2 = 16 * Npix_mid2**2 # Eq 9, in bytes


#Or numerically
print '\nDesign Equation 5a evaluates to:\n'
Fci = 2
print '%s > %.2f GB \tfor SKA1 LOW' % (str(o.McuWork), result4_low / c.giga)
print '%s > %.2f GB \tfor SKA1 MID (Band 1)' % (str(o.McuWork), result4_mid1 / c.giga)
print '%s > %.2f GB \tfor SKA1 MID (Band 2)' % (str(o.McuWork), result4_mid2 / c.giga)

In [ ]:
# Each compute unit must have reasonably fast memory to keep the grids not being immediately worked on (M_(cu,pool )). 
# The experience of ASKAPSoft is that about 10 copies are necessary
o.McuPool > 10 * o.MuvGrid 

In [ ]:
#Or numerically
print '\nDesign Equation 5b evaluates to:\n'
Fci = 2
print '%s > %.2f GB \tfor SKA1 LOW' % (str(o.McuPool), 10*result4_low / c.giga)
print '%s > %.2f GB \tfor SKA1 MID (Band 1)' % (str(o.McuPool), 10*result4_mid1 / c.giga)
print '%s > %.2f GB \tfor SKA1 MID (Band 2)' % (str(o.McuPool), 10*result4_mid2 / c.giga)

In [ ]:
# The FFT and Gridding computing steps are limited by available 
o.NfOut * o.RcuFLOP > 2 * o.Nmajor * (o.Rfft + o.Rrp)  

In [ ]:
bldta = True
otf = False
Nmajor_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.All, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Nmajor', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rfft_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.All, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop_fft', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rrp_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.All, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rrp', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)

Nmajor_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Nmajor', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rfft_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop_fft', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rrp_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rrp', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Nmajor_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Nmajor', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rfft_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rflop_fft', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
Rrp_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rrp', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)




tp = ParameterContainer()
ParameterDefinitions.apply_global_parameters(tp)
ParameterDefinitions.apply_telescope_parameters(tp, Telescopes.SKA1_Low)
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.Continuum)
Nf_out_low = tp.Nf_out
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.Spectral)
Nf_out_low += tp.Nf_out
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.FastImg)
Nf_out_low += tp.Nf_out

tp = ParameterContainer()
ParameterDefinitions.apply_global_parameters(tp)
ParameterDefinitions.apply_telescope_parameters(tp, Telescopes.SKA1_Mid)
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.Continuum)
Nf_out_mid1 = tp.Nf_out
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.Spectral)
Nf_out_mid1 += tp.Nf_out
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.FastImg)
Nf_out_mid1 += tp.Nf_out


tp = ParameterContainer()
ParameterDefinitions.apply_global_parameters(tp)
ParameterDefinitions.apply_telescope_parameters(tp, Telescopes.SKA1_Mid)
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.Continuum)
Nf_out_mid2 = tp.Nf_out
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.Spectral)
Nf_out_mid2 += tp.Nf_out
ParameterDefinitions.apply_imaging_mode_parameters(tp, ImagingModes.FastImg)
Nf_out_mid2 += tp.Nf_out


result5c_low = 2*Nmajor_low*(Rfft_low + Rrp_low)/Nf_out_low
result5c_mid1 = 2*Nmajor_mid*(Rfft_mid1 + Rrp_mid1)/Nf_out_mid1
result5c_mid2 = 2*Nmajor_mid*(Rfft_mid2 + Rrp_mid2)/Nf_out_mid2


#Or numerically
print '\nDesign Equation 5c evaluates to:\n'
print '%s > %.2f TFLOP/s \tfor SKA1 LOW' % (str(o.RcuFLOP), result5c_low / c.tera)
print '%s > %.2f TFLOP/s \tfor SKA1 MID (Band1)' % (str(o.RcuFLOP), result5c_mid1 / c.tera)
print '%s > %.2f TFLOP/s \tfor SKA1 MID (Band2)' % (str(o.RcuFLOP), result5c_mid2 / c.tera)

In [ ]:
# Or, display them as a table
labels = ('Design eqn', '1', '2', '3')
values_1 = ('SKA1-Low', round(result1_low / c.peta, 2), round(result2_low / c.peta, 2), round(result3_low / c.peta, 2))
values_2 = ('SKA1-Mid (Band1)', round(result1_mid1 / c.peta, 2), round(result2_mid1 / c.peta, 2), round(result3_mid1 / c.peta, 2))
values_3 = ('SKA1-Mid (Band2)', round(result1_mid2 / c.peta, 2), round(result2_mid2 / c.peta, 2), round(result3_mid2 / c.peta, 2))

units = ('', 'PFLOP', 'PB/s', 'PB')

iapi.show_table_compare3('"Table 10" in PDR05 v1.85', labels, values_1, values_2, values_3, units)
print 'etc...'

### 6. The total interconnect bandwidth must exceed that needed for faceting

In [ ]:
o.Ncu * o.RcuInter > o.Rinterfacet

In [ ]:
bldta = True
otf = False
Rinterfacet_low = iapi.eval_expression_default(Telescopes.SKA1_Low, ImagingModes.All, Bands.Low, bldta=bldta
                                           , on_the_fly=otf, expression='Rinterfacet', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
RInterfacet_mid1 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid1, bldta=bldta
                                           , on_the_fly=otf, expression='Rinterfacet', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)
RInterfacet_mid2 = iapi.eval_expression_default(Telescopes.SKA1_Mid, ImagingModes.All, Bands.Mid2, bldta=bldta
                                           , on_the_fly=otf, expression='Rinterfacet', scale_predict_by_facet=scale_predict_by_facet, verbose=verbose)




#Or numerically
print '\nDesign Equation for continuum faceting evaluates to:\n'
print '%s > %.2f TB/s \tfor SKA1 LOW, all modes' % (str(o.Ncu * o.RcuInter), Rinterfacet_low / c.tera)
print '%s > %.2f TB/s \tfor SKA1 MID (Band1), all modes' % (str(o.Ncu * o.RcuInter), RInterfacet_mid1 / c.tera)
print '%s > %.2f TB/s \tfor SKA1 MID (Band2), all modes' % (str(o.Ncu * o.RcuInter), RInterfacet_mid2 / c.tera)